In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import talib
import re

warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
idx = pd.IndexSlice

datapath = '../DATA/data.parquet'

In [2]:
DATASTORE = '../DATA/crypto.h5'

In [3]:
with pd.HDFStore(DATASTORE) as store:
    metadata = store['crypto/processed_metadata']
    rdata = store['crypto/raw_data']
    close = store['crypto/raw_data'].Close.unstack('asset')

In [4]:
rdata.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 348507 entries, ('USDCUSDT', datetime.date(2018, 12, 15)) to ('MFTUSDT', datetime.date(2023, 1, 3))
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Open    348507 non-null  float64
 1   High    348507 non-null  float64
 2   Low     348507 non-null  float64
 3   Close   348507 non-null  float64
 4   Volume  348507 non-null  float64
dtypes: float64(5)
memory usage: 14.7+ MB


#### talib indicators

In [4]:
def apply_ta_indicators(group):
    highs_and_lows = {
        'fifty2Week': 364,
        'Quarterly': 364//4,
        'Monthly': 364//12,
        'Weekly': 7
    }

    for key, value in highs_and_lows.items():
        group[f'{key}_Low'] = group.Close.rolling(window=value).min()
        group[f'{key}_High'] = group.Close.rolling(window=value).max()
        
        group[f'percent_above_{key}_low'] = round((((group.Close - group[f'{key}_Low']) / group[f'{key}_Low']) * 100), 2)
        group[f'percent_below_{key}_high'] = round((((group[f'{key}_High'] - group.Close) / group[f'{key}_High']) * 100), 2)

    for key, _ in highs_and_lows.items():
        group = group.drop([f'{key}_Low', f'{key}_High'], axis=1)
    
    window=364
    group['DX'] = talib.DX(group['High'], group['Low'], group['Close'], timeperiod=14) # Directional Flow Index
    group['MFI'] = talib.MFI(group['High'], group.Low, group.Close, group.Volume, timeperiod=14) # Money Flow Index
    group['PPO'] = talib.PPO(group.Close, fastperiod=12, slowperiod=26, matype=0) # 12/26 Day Percent Price Oscillator
    slowk, slowd = talib.STOCH(group.High, group.Low, group.Close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0) # Default settings for stochastic oscillator
    group['slowk'], group['slowd'] = slowk, slowd
    group['Trendline'] = talib.LINEARREG_SLOPE(group.Close, timeperiod=window) # 52-Week Trendline
    group['EMA'] = talib.EMA(group['Close'], timeperiod=14)
    up, mid, low = talib.BBANDS(group['Close'])
    group['BB_up'], group['BB_mid'], group['BB_low'] = up, mid, low
    group['distanceToEMA'] = group['Close'] - group['EMA']

    group.drop(['Close', 'Volume', 'Open', 'High', 'Low'], axis=1, inplace=True)
    
    return group

In [10]:
364//12

30

In [5]:
ta_data = rdata.groupby(level='asset').apply(apply_ta_indicators)

In [6]:
ta_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 348507 entries, ('USDCUSDT', datetime.date(2018, 12, 15)) to ('MFTUSDT', datetime.date(2023, 1, 3))
Data columns (total 19 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   percent_above_fifty2Week_low   227740 non-null  float64
 1   percent_below_fifty2Week_high  227740 non-null  float64
 2   percent_above_Quarterly_low    316942 non-null  float64
 3   percent_below_Quarterly_high   316942 non-null  float64
 4   percent_above_Monthly_low      338241 non-null  float64
 5   percent_below_Monthly_high     338241 non-null  float64
 6   percent_above_Weekly_low       346371 non-null  float64
 7   percent_below_Weekly_high      346371 non-null  float64
 8   DX                             343530 non-null  float64
 9   MFI                            343530 non-null  float64
 10  PPO                            339645 non-null  float64
 11  slowk              

In [ ]:
df.head()

In [ ]:
df.loc['BTCUSDT']['pcs'].tail()

In [ ]:
ta_data.loc['BNBUSDT'].head(16)

#### returns

In [7]:
returns = close.pct_change().stack().swaplevel()

In [8]:
returns = (returns[returns.between(left=returns.quantile(.05), 
                                  right=returns.quantile(.95))].to_frame('returns'))

In [9]:
for col in list(ta_data.columns):
    returns[col] = ta_data[col]

In [ ]:
print(returns.loc['BNBUSDT', 'DX'].head(16), ta_data.loc['BNBUSDT', 'DX'].head(16))

In [ ]:
returns.head()

#### Lagged returns

In [10]:
for t in range(1, 8):
    returns[f't-{t}'] = returns.groupby(level='asset').returns.shift(t)

#### Dummies

In [11]:
dates = pd.to_datetime(returns.index.get_level_values('Date'))

In [12]:
returns['month'] = dates.month
returns['weekday'] = dates.weekday
returns['day'] = dates.day

In [36]:
returns = pd.get_dummies(returns, columns=['month', 'weekday', 'day'])

In [ ]:
returns.info()

##### Data V1 Processing

In [13]:
data_v1 = returns.copy()

In [39]:
print(f'data before dropping na: {len(data_v1)}')
data_v1.dropna(inplace=True)
print(f'data after dropping na: {len(data_v1)}')

data before dropping na: 322976
data after dropping na: 208848


In [14]:
min_obs = 365*0.5
nobs = data_v1.groupby(level='asset').size()
keep = nobs[nobs>min_obs].index
print(f"number of assets before removing minimum obs: {data_v1.index.get_level_values('asset').nunique()}")
data_v1 = data_v1.loc[idx[keep,:], :]
print(f"number of assets after removing minimum obs: {data_v1.index.get_level_values('asset').nunique()}")
data_v1.info()

number of assets before removing minimum obs: 356
number of assets after removing minimum obs: 336
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 321133 entries, ('1INCHUSDT', datetime.date(2020, 12, 28)) to ('ZRXUSDT', datetime.date(2023, 11, 22))
Data columns (total 37 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   returns                        321133 non-null  float64
 1   percent_above_fifty2Week_low   208848 non-null  float64
 2   percent_below_fifty2Week_high  208848 non-null  float64
 3   percent_above_Quarterly_low    285850 non-null  float64
 4   percent_below_Quarterly_high   285850 non-null  float64
 5   percent_above_Monthly_low      303151 non-null  float64
 6   percent_below_Monthly_high     303151 non-null  float64
 7   percent_above_Weekly_low       309296 non-null  float64
 8   percent_below_Weekly_high      309296 non-null  float64
 9   DX                             307182 n

In [15]:
data_v1.head()

returns  percent_above_fifty2Week_low  \
asset     Date                                                 
1INCHUSDT 2020-12-28  0.049755                           NaN   
          2021-01-03  0.080976                           NaN   
          2021-01-04 -0.068863                           NaN   
          2021-01-08 -0.081857                           NaN   
          2021-01-09  0.047527                           NaN   

                      percent_below_fifty2Week_high  \
asset     Date                                        
1INCHUSDT 2020-12-28                            NaN   
          2021-01-03                            NaN   
          2021-01-04                            NaN   
          2021-01-08                            NaN   
          2021-01-09                            NaN   

                      percent_above_Quarterly_low  \
asset     Date                                      
1INCHUSDT 2020-12-28                          NaN   
          2021-01-03                          NaN   
          2021-01-04                          NaN   
          2021-01-08                          NaN   
          2021-01-09                          NaN   

                      percent_below_Quarterly_high  percent_above_Monthly_low  \
asset     Date                                                                  
1INCHUSDT 2020-12-28                           NaN                        NaN   
          2021-01-03                           NaN                        NaN   
          2021-01-04                           NaN                        NaN   
          2021-01-08                           NaN                        NaN   
          2021-01-09                           NaN                        NaN   

                      percent_below_Monthly_high  percent_above_Weekly_low  \
asset     Date                                                               
1INCHUSDT 2020-12-28                         NaN                       NaN   
          2021-01-03                         NaN                     25.27   
          2021-01-04                         NaN                     16.64   
          2021-01-08                         NaN                     15.78   
          2021-01-09                         NaN                     20.49   

                      percent_below_Weekly_high         DX  ...  t-8  t-9  \
asset     Date                                              ...             
1INCHUSDT 2020-12-28                        NaN        NaN  ...  NaN  NaN   
          2021-01-03                      18.75        NaN  ...  NaN  NaN   
          2021-01-04                      24.35        NaN  ...  NaN  NaN   
          2021-01-08                      21.41  23.756997  ...  NaN  NaN   
          2021-01-09                      17.68  23.756997  ...  NaN  NaN   

                      t-10  t-11  t-12  t-13  t-14  month  weekday  day  
asset     Date                                                           
1INCHUSDT 2020-12-28   NaN   NaN   NaN   NaN   NaN     12        0   28  
          2021-01-03   NaN   NaN   NaN   NaN   NaN      1        6    3  
          2021-01-04   NaN   NaN   NaN   NaN   NaN      1        0    4  
          2021-01-08   NaN   NaN   NaN   NaN   NaN      1        4    8  
          2021-01-09   NaN   NaN   NaN   NaN   NaN      1        5    9  

[5 rows x 37 columns]

In [16]:
with pd.HDFStore(DATASTORE) as store:
    store.put('crypto/data_v3', data_v1)

In [43]:
with pd.HDFStore(DATASTORE) as store:
    store.put('crypto/data_v1', data_v1)

#### Factorized Data

In [7]:
f_returns = close.pct_change().stack().swaplevel()

In [9]:
f_returns = (f_returns[f_returns.between(left=f_returns.quantile(.05), 
                                  right=f_returns.quantile(.95))].to_frame('returns'))

In [10]:
for col in list(ta_data.columns):
    f_returns[col] = ta_data[col]

In [11]:
for t in range(1, 15):
    f_returns[f't-{t}'] = f_returns.groupby(level='asset').returns.shift(t)

In [ ]:
def factorize_cats(cats)

### Get Crypto Xtics through metadata

In [44]:
metadata.market_cap.head()

asset
BTCUSDT    714288494324
ETHUSDT    238817767826
BNBUSDT     37749200414
XRPUSDT     33423459069
SOLUSDT     25160865365
Name: market_cap, dtype: int64

In [45]:
metadata['size'] = pd.qcut(metadata.market_cap, q=10, labels=list(range(1, 11)))
metadata.drop(['market_cap'], axis=1, inplace=True)

In [46]:
metadata = metadata[['size', 'category']]
metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 356 entries, BTCUSDT to OAXUSDT
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   size      356 non-null    category
 1   category  356 non-null    object  
dtypes: category(1), object(1)
memory usage: 6.3+ KB


#### create dummies for size

In [47]:
metadata = pd.get_dummies(metadata, columns=['size'],
                         prefix = ['size'],
                         prefix_sep=['_'])
metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 356 entries, BTCUSDT to OAXUSDT
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  356 non-null    object
 1   size_1    356 non-null    uint8 
 2   size_2    356 non-null    uint8 
 3   size_3    356 non-null    uint8 
 4   size_4    356 non-null    uint8 
 5   size_5    356 non-null    uint8 
 6   size_6    356 non-null    uint8 
 7   size_7    356 non-null    uint8 
 8   size_8    356 non-null    uint8 
 9   size_9    356 non-null    uint8 
 10  size_10   356 non-null    uint8 
dtypes: object(1), uint8(10)
memory usage: 9.0+ KB


#### create dummies for categories

In [48]:
cat_encoding = metadata.category.str.get_dummies(',')

In [49]:
metadata = pd.concat([metadata, cat_encoding], axis=1)

In [50]:
pd.set_option('display.max_columns', None)

In [51]:
metadata.drop(['category'], axis=1, inplace=True)
metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 356 entries, BTCUSDT to OAXUSDT
Columns: 173 entries, size_1 to zksync-ecosystem
dtypes: int64(163), uint8(10)
memory usage: 459.6+ KB


#### Combine metadata and returns

In [52]:
data = (returns
       .reset_index('Date')
       .merge(metadata, left_index=True, right_index=True)
       .dropna()
       .set_index('Date', append=True))

s = len(returns.columns)

data.iloc[:, s:] = data.iloc[:, s:].astype(int).apply(pd.to_numeric, downcast='integer')
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 208848 entries, ('1INCHUSDT', datetime.date(2021, 12, 24)) to ('ZRXUSDT', datetime.date(2023, 11, 22))
Columns: 257 entries, returns to zksync-ecosystem
dtypes: float64(34), int8(173), uint8(50)
memory usage: 99.5+ MB


In [53]:
data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 208848 entries, ('1INCHUSDT', datetime.date(2021, 12, 24)) to ('ZRXUSDT', datetime.date(2023, 11, 22))
Columns: 257 entries, returns to zksync-ecosystem
dtypes: float64(34), int8(173), uint8(50)
memory usage: 99.5+ MB


In [ ]:
data.head()

In [54]:
data.index.get_level_values('asset').nunique()

307

In [55]:
min_obs = 365*0.5
nobs = data.groupby(level='asset').size()
keep = nobs[nobs>min_obs].index

data = data.loc[idx[keep,:], :]
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 207362 entries, ('1INCHUSDT', datetime.date(2021, 12, 24)) to ('ZRXUSDT', datetime.date(2023, 11, 22))
Columns: 257 entries, returns to zksync-ecosystem
dtypes: float64(34), int8(173), uint8(50)
memory usage: 98.8+ MB


In [56]:
data.index.get_level_values('asset').nunique()

288

In [58]:
data.loc['BTCUSDT', 'DX'].head()

Date
2018-08-15    24.285306
2018-08-16    24.285306
2018-08-17    18.055109
2018-08-18    15.968290
2018-08-19    15.968290
Name: DX, dtype: float64

In [59]:
with pd.HDFStore(DATASTORE) as store:
    store.put('crypto/data_v2', data)

In [61]:
for x, y in zip(data.loc['BTCUSDT', 'Trendline'].head(20), ta_data.loc['BTCUSDT', 'Trendline'].dropna()):
    if x != y:
        print(x, y)

In [62]:
with pd.HDFStore(DATASTORE) as store:
    store.remove('crypto/engineered_features')

In [63]:
with pd.HDFStore(DATASTORE) as store:
    print(store.info())

<class 'pandas.io.pytables.HDFStore'>
File path: ../DATA/crypto.h5
/crypto/categories                    frame        (shape->[1,1])       
/crypto/data                          frame                             
/crypto/data_v1                       frame        (shape->[207362,84]) 
/crypto/data_v2                       frame        (shape->[207362,257])
/crypto/metadata                      frame        (shape->[1,6])       
/crypto/processed_metadata            frame        (shape->[356,7])     
/crypto/raw_data                      frame        (shape->[348507,5])  
